# DESCRITPTION:
The following code contains the implementation to generate a sample from the entire dataset. Currentlyy, we have chosen the 

In [ ]:
import pandas as pd
import numpy as np
import re
import json
from time import time

In [ ]:
def convertDate(date):
    date = str(date)
    date = re.sub(pattern='[.]\d* ', repl=' ', string=date)
    new_date = pd.to_datetime(date, infer_datetime_format=True, utc=True).timestamp()
    # format = "%Y-%m-%d %H-%M-%S %Z"

    return new_date

In [ ]:
def convertLevel(level):
    if level == "Level 1" or "Level1":
        return 1
    elif level == "Level 5" or "Level5":
        return 1
    elif level == "Level B1" or "LevelB1":
        return 0
    else:
        return level

In [ ]:
def splitTime(dates):
    new_dates = []
    new_times = []
    new_timestamps = []

    for date in dates:
        new_value = pd.to_datetime(date, infer_datetime_format=True, utc=True)
        new_dates.append(new_value.date())
        new_times.append(new_value.time())
        new_timestamps.append(new_value.timestamp())

    new_df = pd.DataFrame({'date': new_dates,
                           'time': new_times,
                           'tmstmp': new_timestamps,
                           'localtime': dates})

    return new_df

In [ ]:
def process_chunk(df):

    # filter for current date
    df = df.loc[df['localtime'].str.contains('2018-06-09')]
    df = pd.concat([df, splitTime(df['localtime'])], axis=1)
    df = df.drop(columns=['localtime'], axis=1)
    df['time'] = df.loc[df['time'].str.split(":", expand=True)]

    if len(global_macid_dict.keys()) > 0:
        unique_macids = set(df.ClientMacAddr.unique()).difference(set(global_macid_dict.keys()))
    else:
        unique_macids = set(df.ClientMacAddr.unique())

    start = max(global_macid_revers_dict.keys(), default=0)

    for index, value in enumerate(unique_macids, start=start+1):
        global_macid_dict[value] = index
        global_macid_revers_dict[index] = value

    df['ClientMacAddr'] = df['ClientMacAddr'].map(global_macid_dict)
    df.to_csv(output_file_path, mode='a', index=False)

    with open(mapid_map_path", "a+") as fp:
        json.dump(global_macid_dict, fp, indent=4)
        fp.close()

In [ ]:
# The following pararmeters are configurable and can be changed as per the business requirements
input_file_path = ".\prometheus-whole.csv"
output_file_path = ".\prometheus-one-day.csv"
mapid_map_path = ".\macid-map.txt"

In [ ]:
global_macid_dict = {}
global_macid_revers_dict = {}

cols = ['Level', 'ClientMacAddr', 'lat', 'lng', 'localtime']
chunksize = 10**6

for chunk in pd.read_csv(input_file_path,
                         usecols=['Level', 'ClientMacAddr', 'lat', 'lng', 'localtime'],
                         converters={'Level': convertLevel}, chunksize=chunksize):
    process_chunk(chunk)

print("Sample Generation Completed")